to recap, the pipeline and its associated terminology will be

message $m \in \mathbb{C}^{N/2}$ $\rightarrow_{\text{encode}}$ plaintext $p(X) \in \mathbb{Z}[X]/(X^N + 1)$ $\rightarrow_{\text{encrypt}}$ ciphertext $c = (c_0(X), c_1(X)) \in (\mathbb{Z}_q[X](X^N + 1))^2$

Then the function of interest $f$ will be homomorphically evaluated upon the ciphertext $c$ to get

new message $m' = f(m) \in \mathbb{C}^{N/2}$ $\leftarrow_{\text{decode}}$ new plaintext $p'(X) = f(p(X)) \in \mathbb{Z}[X]/(X^N + 1)$ $\leftarrow_{\text{decrypt}}$ new ciphertext $c' = f(c) \in (\mathbb{Z}_q[X](X^N + 1))^2$




In [31]:
import numpy as np
from numpy.linalg import inv
from numpy.polynomial import Polynomial

In [2]:
N = 4
M = 2*N
# the 1st Mth root of unity
xi = np.exp(2 * 1j * np.pi / M)

In [57]:
xi

(0.7071067811865476+0.7071067811865475j)

In [3]:
# the message in $\mathbb{C}^{N}$ that we are encoding
z = [1, 2, 3, 4]
assert len(z) == N

In [69]:
# somewhat awkwardly written, but the way to think of this is that we want a 1 for the constant term and then another 1 at the degree N position
# so that this is the polynomial X^N + 1
SUBGROUP_POLY = Polynomial([1] + ([0] * (N-1)) + [1] )
SUBGROUP_POLY

Polynomial([1., 0., 0., 0., 1.], domain=[-1,  1], window=[-1,  1], symbol='x')

In [70]:
A = np.vander([xi ** ((2*i) - 1) for i in range(1, N+1)], increasing=True)

def encode(message):
    '''Encodes a message in C^N to a plaintext in C[X]/(X^N + 1).'''
    z = np.matrix(message).reshape((N, 1))
    alpha = (inv(A) @ z)
    # need to transform z to a list since an np.matrix is definitionally 2d and cannot be squeezed
    alpha = np.squeeze(alpha.tolist(), axis=1)
    return Polynomial(alpha)

def decode(polynomial):
    '''Decodes a message from C[X]/(X^N + 1) to C^N by evaluating it on the odd Mth roots of unity'''
    return [polynomial(xi ** ((2*i) - 1)) for i in range(1, N+1)]

In [71]:
A

array([[ 1.00000000e+00+0.j        ,  7.07106781e-01+0.70710678j,
         2.22044605e-16+1.j        , -7.07106781e-01+0.70710678j],
       [ 1.00000000e+00+0.j        , -7.07106781e-01+0.70710678j,
        -4.44089210e-16-1.j        ,  7.07106781e-01+0.70710678j],
       [ 1.00000000e+00+0.j        , -7.07106781e-01-0.70710678j,
         1.11022302e-15+1.j        ,  7.07106781e-01-0.70710678j],
       [ 1.00000000e+00+0.j        ,  7.07106781e-01-0.70710678j,
        -1.38777878e-15-1.j        , -7.07106781e-01-0.70710678j]])

In [72]:
m = encode(z)

In [73]:
m

Polynomial([ 2.50000000e+00+6.52256027e-16j, -3.88578059e-16+7.07106781e-01j,
       -4.02455846e-16+5.00000000e-01j, -7.77156117e-16+7.07106781e-01j], domain=[-1,  1], window=[-1,  1], symbol='x')

In [68]:
decoded_z = decode(m)

[(1+9.714451465470159e-17j),
 (2-1.2490009027032972e-16j),
 (3-4.1633363423442976e-17j),
 (3.9999999999999996+3.191891195797329e-16j)]

In [74]:
# test of homomorphic addition/multiplication

m1 = [1, 2, 3, 4]
m2 = [-1, -2, -3, -4]

z1, z2 = encode(m1), encode(m2)

result1 = decode(z1 + z2)
print(result1) # expect [0, 0, 0, 0]
result2 = decode(z1 * z2 % SUBGROUP_POLY)
print(result2) # expect [-1, -4, -9, -16]

[0j, 0j, 0j, 0j]
[(-1-1.3155301742304191e-15j), (-4-4.273517545302939e-16j), (-9-2.31473089639306e-15j), (-16-7.088689902281233e-15j)]
